In [3]:
import numpy as np
import tensorflow as tf
import gym 
from DDPG import DDPG
import matplotlib.pyplot as plt

In [6]:
import qutip as qt
from qutip import tensor, basis, projection, identity, mesolve


class STIRAP:
    def __init__(self, N_timesteps, step_duration, step_resolution):
        self.N_timesteps=N_timesteps
        self.resolution=step_resolution
        self.duration=step_duration
        self.Omega=1
        self.delta2_0=50
        self.deltaP_0=50
        self.times = np.linspace(0,N_timesteps*step_duration,N_timesteps*step_resolution)
        self.T=self.times[-1]
        self.full_state_his = []
        
    def build_hamiltonian(self, deltaP, delta2):
        H = (deltaP*qt.projection(3,1,1) + delta2*qt.projection(3,2,2) + 
             0.5*self.Omega*(projection(3,0,1) + projection(3,1,2)
                        + projection(3,2,1) + projection(3,1,0))
            )
        return H
    
    def reset(self):
        self.state=basis(3,0)
        self.done=False
        return self.state.full().flatten()
    
    def step(self, actions, alpha=1, beta=0.5):
        inner_times = np.linspace(0,self.duration, self.resolution)
        deltaP=actions[0]*self.deltaP_0
        delta2=actions[1]*self.deltaP_0
        H = self.build_hamiltonian(deltaP, delta2)
        res = mesolve(H=H, tlist=inner_times, rho0=self.state)
        self.full_state_his.extend(res.states[:])
        final_state = res.states[-1]
        self.state=final_state
        reward = alpha*np.abs(final_state.full().flatten()[2])**2 
#         - beta*np.abs(final_state.full().flatten()[1])**2
        if reward>0.99:
            self.done=True
        else:
            pass
        info={}
        return final_state.full().flatten(), reward, self.done, info
   
    def close(self):
        return
    def render(self):
        return

In [7]:
env = STIRAP(N_timesteps=20, step_resolution=10, step_duration=4)
acts = 2
state_size = 3



ddpg = DDPG(n_actions=acts, state_size=state_size, learning_batch_size=32, 
            act_struct=[32,32], crit_struct=[32,32])

num_episodes =10000
returns_array = np.zeros(num_episodes)
counter = 0
episode_length=env.N_timesteps
training_start=1000#batch_size*5
returns_benchmark = 0
for episode in range(num_episodes):
    state = env.reset()
    done=False
    returns=0.
    sigma = ddpg.sigma_scheduler(episode=episode, num_episodes=num_episodes)
    t=0
    while not done:
        counter+=1
        main_actor_prediction = ddpg.main_actor(state.reshape((1,state_size)))
        noisy_predict = ddpg.get_noisy_output(main_actor_prediction, sig=sigma)
        action = 2*noisy_predict
        next_state, reward, done, _ = env.step(action.numpy())
        t+=1
        if t==episode_length:
            done=True
        returns+=reward
        ddpg.store_sample(state, action, next_state, reward, done)
        state=next_state

        if counter > training_start:
            states, actions, rewards, next_states, dones = ddpg.sample_experiences()
            ddpg.train_critic(states=states, rewards=rewards, next_states=next_states, dones=dones)
            ddpg.train_actor(states=states)
            ddpg.soft_update(tau=0.05)
    returns_array[episode] = returns
    if returns > returns_benchmark:
        best_actor = ddpg.main_actor
#         best_actor.save("DDPG-Actor-v1-Best".format(episode), include_optimizer=False)
        returns_benchmark = returns
        print("episode: {}, Best Return: {}".format(episode,returns))
    if episode%25==0:
#         main_actor.save("DDPG-Actor-v0-episode-{}".format(episode), include_optimizer=False)
        print("episode: {}, Return: {}".format(episode,returns))

episode: 0, Best Return: 3.0921881689972186
episode: 0, Return: 3.0921881689972186
episode: 3, Best Return: 6.471324951040243
episode: 25, Return: 0.025547640006487304
episode: 50, Return: 0.07854903175552824
episode: 75, Return: 2.629944296481004e-08
episode: 100, Return: 1.9735022258051838e-08
episode: 125, Return: 1.9609681661748552e-08
episode: 150, Return: 2.1084569653672165e-08
episode: 175, Return: 2.9388509363896165e-08
episode: 200, Return: 2.1564084919009354e-08
episode: 225, Return: 2.430692719182074e-08
episode: 250, Return: 2.23100787252512e-08
episode: 275, Return: 2.710010842639585e-08
episode: 300, Return: 2.045287323734289e-08
episode: 325, Return: 2.2589585408040397e-08
episode: 350, Return: 2.658465778990319e-08
episode: 375, Return: 2.4616512696716692e-08
episode: 400, Return: 2.5340537739304706e-08
episode: 425, Return: 2.1430508236815275e-08
episode: 450, Return: 1.9436344888274538e-08
episode: 475, Return: 2.0679625155743733e-08
episode: 500, Return: 1.7904609660

KeyboardInterrupt: 

In [ ]:
actions=np.zeros((2,env.N_timesteps))
state = env.reset()
done=False
returns=0.
sigma = ddpg.sigma_scheduler(episode=episode, num_episodes=num_episodes)
t=0
while not done:
    counter+=1
    main_actor_prediction = best_actor(state.reshape((1,state_size)))
    noisy_predict = ddpg.get_noisy_output(main_actor_prediction, sig=0)
    action = noisy_predict
    next_state, reward, done, _ = env.step(action.numpy())
    actions[:,t]=action.numpy()[:]
    t+=1
    if t==episode_length:
        done=True
    state=next_state

In [ ]:
plt.step(np.arange(env.N_timesteps), actions[0])
plt.step(np.arange(env.N_timesteps), actions[1])

In [ ]:
populations = np.zeros((3,env.times.shape[0]))
for i, t in enumerate(env.times):
    populations[:,i]=np.abs(env.full_state_his[i].full().flatten()[:])**2
    
plt.plot(env.times, populations[0,:], label="|0>")
plt.plot(env.times, populations[1,:], label="|1>")
plt.plot(env.times, populations[2,:], label="|2>")
plt.legend()
    

In [159]:
noisy_predict

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 0.84042394, -1.        ], dtype=float32)>